<a href="https://colab.research.google.com/github/ronak88/Sentiment-analysis-stock-based-dataset/blob/main/Stock_Sentiment_Analysis_Report_for_top_five_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from joblib import dump,load

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ml Data/Data.csv', encoding="ISO-8859-1")
# https://www.kaggle.com/aaron7sun/stocknews (data link)

In [ ]:
df.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,"Southgate strikes, Leeds pay the penalty",Hammers hand Robson a youthful lesson,Saints party like it's 1999,Wear wolves have turned into lambs,Stump mike catches testy Gough's taunt,Langer escapes to hit 167,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,Has Cubie killed fees?,A tale of two tails,I say what I like and I like what I say,"Elbows, Eyes and Nipples",Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite


In [ ]:
train_data = df[df['Date']<'20150101']
test_data = df[df['Date']>'20141231']


In [ ]:
train_feature = train_data.iloc[:,2:27]
train_feature.replace("[^a-zA-Z]"," ",regex=True,inplace=True)

test_feature = test_data.iloc[:,2:27]
test_feature.replace("[^a-zA-Z]"," ",regex=True,inplace=True)

In [ ]:
list1=[i for i in range(25)]
new_index= [str(i) for i in list1]
train_feature.columns = new_index
test_feature.columns = new_index
test_feature.head(2)
# train_feature.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
3723,Most cases of cancer are the result of sheer b...,Iran dismissed United States efforts to fight ...,Poll One in Germans would join anti Muslim ...,UK royal family s Prince Andrew named in US la...,Some asylum seekers refused to leave the bu...,Pakistani boat blows self up after India navy ...,Sweden hit by third mosque arson attack in a week,cars set alight during French New Year,Salaries for top CEOs rose twice as fast as av...,Norway violated equal pay law judge says Jud...,Imam wants radical recruiters of Muslim youth ...,Saudi Arabia beheaded people in the m...,A living hell for slaves on remote South Kor...,Worlds richest get richer adding bn in...,Rental Car Stereos Infringe Copyright Music R...,Ukrainian minister threatens TV channel with c...,Palestinian President Mahmoud Abbas has entere...,Israeli security center publishes names of ...,The year was the deadliest year yet in Sy...,A Secret underground complex built by the Nazi...,Restrictions on Web Freedom a Major Global Iss...,Austrian journalist Erich Mchel delivered a pr...,Thousands of Ukraine nationalists march in Kiev,Chinas New Years Resolution No More Harvestin...,Authorities Pull Plug on Russia s Last Politic...
3724,Moscow gt Beijing high speed train will reduc...,Two ancient tombs were discovered in Egypt on ...,China complains to Pyongyang after N Korean so...,Scotland Headed Towards Being Fossil Fuel Free...,Prime Minister Shinzo Abe said Monday he will ...,Sex slave at centre of Prince Andrew scandal f...,Gay relative of Hamas founder faces deportatio...,The number of female drug addicts in Iran has ...,After Decades of Searching the Causeway for t...,India lost tigers in,Lizard Squad hacking of Sony was huge marketi...,Russia may face chaos if extra sanctions imp...,The Saudi air and land forces carry out an att...,ISIL is reportedly setting up a bank in attemp...,Israel to brand anti assimilation group Lehava...,The Islamic State has approved a budget o...,Iceland To Withdraw EU Application Lift Capit...,Blackfield Capital Founder Goes Missing The v...,Rocket stage crashes back to Earth in rural Ch...,Dead as Aircraft Bombs Greek Tanker in Libya...,Belgian murderer Frank Van Den Bleeken to die ...,Czech President criticizes Ukrainian PM says ...,Vietnamese jets join search for missing F...,France seeks end to Russia sanctions over Ukraine,China scraps rare earths caps


In [ ]:
for column in new_index:
    train_feature[column]= train_feature[column].str.lower()
for index in new_index:
    test_feature[index]= test_feature[index].str.lower()

In [ ]:
train_feature.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,a hindrance to operations extracts from the...,scorecard,hughes instant hit buoys blues,jack gets his skates on at ice cold alex,chaos as maracana builds up for united,depleted leicester prevail as elliott spoils e...,hungry spurs sense rich pickings,gunners so wide of an easy target,derby raise a glass to strupar s debut double,southgate strikes leeds pay the penalty,hammers hand robson a youthful lesson,saints party like it s,wear wolves have turned into lambs,stump mike catches testy gough s taunt,langer escapes to hit,flintoff injury piles on woe for england,hunters threaten jospin with new battle of the...,kohl s successor drawn into scandal,the difference between men and women,sara denver nurse turned solicitor,diana s landmine crusade put tories in a panic,yeltsin s resignation caught opposition flat f...,russian roulette,sold out,recovering a title
1,scorecard,the best lake scene,leader german sleaze inquiry,cheerio boyo,the main recommendations,has cubie killed fees,has cubie killed fees,has cubie killed fees,hopkins furious at foster s lack of hannibal...,has cubie killed fees,a tale of two tails,i say what i like and i like what i say,elbows eyes and nipples,task force to assess risk of asteroid collision,how i found myself at last,on the critical list,the timing of their lives,dear doctor,irish court halts ira man s extradition to nor...,burundi peace initiative fades after rebels re...,pe points the way forward to the ecb,campaigners keep up pressure on nazi war crime...,jane ratcliffe,yet more things you wouldn t know without the ...,millennium bug fails to bite


In [ ]:
test_feature.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
3723,most cases of cancer are the result of sheer b...,iran dismissed united states efforts to fight ...,poll one in germans would join anti muslim ...,uk royal family s prince andrew named in us la...,some asylum seekers refused to leave the bu...,pakistani boat blows self up after india navy ...,sweden hit by third mosque arson attack in a week,cars set alight during french new year,salaries for top ceos rose twice as fast as av...,norway violated equal pay law judge says jud...,imam wants radical recruiters of muslim youth ...,saudi arabia beheaded people in the m...,a living hell for slaves on remote south kor...,worlds richest get richer adding bn in...,rental car stereos infringe copyright music r...,ukrainian minister threatens tv channel with c...,palestinian president mahmoud abbas has entere...,israeli security center publishes names of ...,the year was the deadliest year yet in sy...,a secret underground complex built by the nazi...,restrictions on web freedom a major global iss...,austrian journalist erich mchel delivered a pr...,thousands of ukraine nationalists march in kiev,chinas new years resolution no more harvestin...,authorities pull plug on russia s last politic...
3724,moscow gt beijing high speed train will reduc...,two ancient tombs were discovered in egypt on ...,china complains to pyongyang after n korean so...,scotland headed towards being fossil fuel free...,prime minister shinzo abe said monday he will ...,sex slave at centre of prince andrew scandal f...,gay relative of hamas founder faces deportatio...,the number of female drug addicts in iran has ...,after decades of searching the causeway for t...,india lost tigers in,lizard squad hacking of sony was huge marketi...,russia may face chaos if extra sanctions imp...,the saudi air and land forces carry out an att...,isil is reportedly setting up a bank in attemp...,israel to brand anti assimilation group lehava...,the islamic state has approved a budget o...,iceland to withdraw eu application lift capit...,blackfield capital founder goes missing the v...,rocket stage crashes back to earth in rural ch...,dead as aircraft bombs greek tanker in libya...,belgian murderer frank van den bleeken to die ...,czech president criticizes ukrainian pm says ...,vietnamese jets join search for missing f...,france seeks end to russia sanctions over ukraine,china scraps rare earths caps


In [ ]:
train_headlines = []
for row in range(0,len(train_feature.index)):
    train_headlines.append(' '.join(str(x) for x in train_feature.iloc[row,0:25]))

In [ ]:
test_headlines = []
for row in range(0,len(test_feature.index)):
    test_headlines.append(' '.join(str(x) for x in test_feature.iloc[row,0:25]))

In [ ]:
train_headlines[0]

'a  hindrance to operations   extracts from the leaked reports scorecard hughes  instant hit buoys blues jack gets his skates on at ice cold alex chaos as maracana builds up for united depleted leicester prevail as elliott spoils everton s party hungry spurs sense rich pickings gunners so wide of an easy target derby raise a glass to strupar s debut double southgate strikes  leeds pay the penalty hammers hand robson a youthful lesson saints party like it s      wear wolves have turned into lambs stump mike catches testy gough s taunt langer escapes to hit     flintoff injury piles on woe for england hunters threaten jospin with new battle of the somme kohl s successor drawn into scandal the difference between men and women sara denver  nurse turned solicitor diana s landmine crusade put tories in a panic yeltsin s resignation caught opposition flat footed russian roulette sold out recovering a title'

In [ ]:
test_headlines[4]

'new charlie hebdo issue to come out next week   it s very hard  we are all suffering  with grief  with fear  but we will do it anyway because stupidity will not win   ahmed merabet  cop killed in paris attacks  was muslim youngest suspect in charlie hebdo attack turns himself in grenades thrown at a mosque in le mans  west of paris danish newspaper to print charlie hebdo cartoons on islam muslim politician from india offers   m to charlie hebdo attackers vladimir putin has condemned a deadly terrorist attack in paris  and confirmed moscows readiness to continue cooperation with france in battling terrorism  putin also expressed his condolences to the victims  families and wished the injured a speedy recovery  turkey is trying to hide any evidence that holds turkish leaders responsible for the support of terrorist groups  especially the islamic state of iraq and syria  isis   turkish media is reporting  danish paper that published    muhammad cartoons boosts security two charlie hebdo 

In [ ]:
# sentences = nltk.sent_tokenize(train_headlines[0])
# sentences

In [ ]:
wordnet = WordNetLemmatizer()
ps = PorterStemmer()

In [ ]:
train_headlines1=[]
for i in range(len(train_headlines)):
    paragrap = train_headlines[i].lower()
    words = paragrap.split()
#     words = [wordnet.lemmatize(word) for word in words if not word in set(stopwords.words('english'))]
#     words = [ps.stem(word) for word in words if not word in set(stopwords.words('english'))]
    words = [ word for word in words if not word in set(stopwords.words('english'))]
    paragrap = ' '.join(words)
    train_headlines1.append(paragrap)
    

In [ ]:
train_headlines1[0]

'hindrance operations extracts leaked reports scorecard hughes instant hit buoys blues jack gets skates ice cold alex chaos maracana builds united depleted leicester prevail elliott spoils everton party hungry spurs sense rich pickings gunners wide easy target derby raise glass strupar debut double southgate strikes leeds pay penalty hammers hand robson youthful lesson saints party like wear wolves turned lambs stump mike catches testy gough taunt langer escapes hit flintoff injury piles woe england hunters threaten jospin new battle somme kohl successor drawn scandal difference men women sara denver nurse turned solicitor diana landmine crusade put tories panic yeltsin resignation caught opposition flat footed russian roulette sold recovering title'

In [ ]:
test_headlines1=[]
for i in range(len(test_headlines)):
    paragrap = test_headlines[i].lower()
    words = paragrap.split()
#     words = [wordnet.lemmatize(word) for word in words if not word in set(stopwords.words('english'))]
#     words = [ps.stem(word) for word in words if not word in set(stopwords.words('english'))]
    words = [ word for word in words if not word in set(stopwords.words('english'))]
    paragrap = ' '.join(words)
    test_headlines1.append(paragrap)


In [ ]:
# Training freature

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

cv = CountVectorizer()
tfid = TfidfVectorizer()
x_train_cv_features = cv.fit_transform(train_headlines1).toarray()
x_train_tfid_features = tfid.fit_transform(train_headlines1).toarray() 


In [ ]:
# Testing feature

x_test_cv_features = cv.transform(test_headlines1).toarray()
x_test_tfid_features = tfid.transform(test_headlines1).toarray()


**1. RandomForestClassifier**

---



In [ ]:
# RandomForestClassifier Model importing

from sklearn.ensemble import RandomForestClassifier

randomclassifier_Cv = RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier_Tfid = RandomForestClassifier(n_estimators=200,criterion='entropy')

In [ ]:
# RandomForestClassifier Model training

randomclassifier_Cv.fit(x_train_cv_features,train_data['Label'])
randomclassifier_Tfid.fit(x_train_tfid_features,train_data['Label'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
dump(randomclassifier_Cv,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/randomclassifier_Cv.joblib')
dump(randomclassifier_Tfid,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/randomclassifier_Tfid.joblib')

['/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/randomclassifier_Tfid.joblib']

In [ ]:
# RandomForestClassifier Model prediction

Cv_predictions_1 = randomclassifier_Cv.predict(x_test_cv_features)
Tfid_predictions_1 = randomclassifier_Tfid.predict(x_test_tfid_features)


In [ ]:
matrix_1 = confusion_matrix(test_data['Label'],Cv_predictions_1)
report_1 = classification_report(test_data['Label'],Cv_predictions_1)
print(matrix_1)
print("-----------------------------classification_report RandomForestClassifier (using bag-of-words ) ----------------------------------------------\n")
print(report_1)
print("\n------------------------------------------------------------------------------------------------- ----------------------------------------------\n")
matrix_1 = confusion_matrix(test_data['Label'],Tfid_predictions_1)
report_1 = classification_report(test_data['Label'],Tfid_predictions_1)
print(matrix_1)
print("----------------------------Classification Report RandomForestClassifier (using TF_ID)----------------------------------------------\n")
print(report_1)

[[146  40]
 [ 19 173]]
-----------------------------classification_report RandomForestClassifier (using bag-of-words ) ----------------------------------------------

              precision    recall  f1-score   support

           0       0.88      0.78      0.83       186
           1       0.81      0.90      0.85       192

    accuracy                           0.84       378
   macro avg       0.85      0.84      0.84       378
weighted avg       0.85      0.84      0.84       378


------------------------------------------------------------------------------------------------- ----------------------------------------------

[[147  39]
 [ 21 171]]
----------------------------Classification Report RandomForestClassifier (using TF_ID)----------------------------------------------

              precision    recall  f1-score   support

           0       0.88      0.79      0.83       186
           1       0.81      0.89      0.85       192

    accuracy                          

**2. KNeighborsClassifier**

---


In [ ]:
# KNeighborsClassifier Model importing and training

from sklearn.neighbors import KNeighborsClassifier
KNNclassifier_Cv = KNeighborsClassifier(n_neighbors=6)
KNNclassifier_Tfid = KNeighborsClassifier(n_neighbors=6)

KNNclassifier_Cv.fit(x_train_cv_features,train_data['Label'])
KNNclassifier_Tfid.fit(x_train_tfid_features,train_data['Label'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [ ]:
dump(KNNclassifier_Cv,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/KNNclassifier_Cv.joblib')
dump(KNNclassifier_Tfid,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/KNNclassifier_Tfid.joblib')

['/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/KNNclassifier_Tfid.joblib']

In [ ]:
# KNeighborsClassifier Model prediction

Cv_predictions_2 = KNNclassifier_Cv.predict(x_test_cv_features)
Tfid_predictions_2 = KNNclassifier_Tfid.predict(x_test_tfid_features)

In [ ]:
matrix_2 = confusion_matrix(test_data['Label'],Cv_predictions_2)
report_2 = classification_report(test_data['Label'],Cv_predictions_2)
print(matrix_2)
print("-----------------------------classification Report KNeighborsClassifier Model (using bag-of-words ) ----------------------------------------------\n")
print(report_2)
print("\n------------------------------------------------------------------------------------------------- ----------------------------------------------\n")
matrix_2 = confusion_matrix(test_data['Label'],Tfid_predictions_2)
report = classification_report(test_data['Label'],Tfid_predictions_2)
print(matrix_2)
print("----------------------------Classification Report RandomForestClassifier (using TF_ID)----------------------------------------------\n")
print(report_2)

[[ 28 158]
 [  3 189]]
-----------------------------classification Report KNeighborsClassifier Model (using bag-of-words ) ----------------------------------------------

              precision    recall  f1-score   support

           0       0.90      0.15      0.26       186
           1       0.54      0.98      0.70       192

    accuracy                           0.57       378
   macro avg       0.72      0.57      0.48       378
weighted avg       0.72      0.57      0.48       378


------------------------------------------------------------------------------------------------- ----------------------------------------------

[[132  54]
 [ 84 108]]
----------------------------Classification Report RandomForestClassifier (using TF_ID)----------------------------------------------

              precision    recall  f1-score   support

           0       0.90      0.15      0.26       186
           1       0.54      0.98      0.70       192

    accuracy                      

**3. SVM**

---



In [ ]:
from sklearn import svm
SVMclf_Cv = svm.SVC()
SVMclf_Tfid = svm.SVC()

SVMclf_Cv.fit(x_train_cv_features,train_data['Label'])
SVMclf_Tfid.fit(x_train_tfid_features,train_data['Label'])

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
dump(SVMclf_Cv,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/SVMclf_Cv.joblib')
dump(SVMclf_Tfid,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/SVMclf_Tfid.joblib')

['/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/SVMclf_Tfid.joblib']

In [ ]:
# SVM Model prediction

Cv_predictions_3 = SVMclf_Cv.predict(x_test_cv_features)
Tfid_predictions_3 = SVMclf_Tfid.predict(x_test_tfid_features)

In [ ]:
matrix_3 = confusion_matrix(test_data['Label'],Cv_predictions_3)
report_3 = classification_report(test_data['Label'],Cv_predictions_3)
print(matrix_3)
print("----------------------------classification_report Support Vector Machines Model (using bag-of-words ) ----------------------------------------------\n")
print(report_3)
print("\n------------------------------------------------------------------------------------------------------------------------------------------------\n")
matrix_3 = confusion_matrix(test_data['Label'],Tfid_predictions_3)
report_3 = classification_report(test_data['Label'],Tfid_predictions_3)
print(matrix_3)
print("----------------------------Classification Report Support Vector Machines Model (using TF_ID)-----------------------------------------------------------\n")
print(report_3)

[[134  52]
 [ 11 181]]
----------------------------classification_report Support Vector Machines Model (using bag-of-words ) ----------------------------------------------

              precision    recall  f1-score   support

           0       0.92      0.72      0.81       186
           1       0.78      0.94      0.85       192

    accuracy                           0.83       378
   macro avg       0.85      0.83      0.83       378
weighted avg       0.85      0.83      0.83       378


------------------------------------------------------------------------------------------------------------------------------------------------

[[132  54]
 [  4 188]]
----------------------------Classification Report Support Vector Machines Model (using TF_ID)-----------------------------------------------------------

              precision    recall  f1-score   support

           0       0.97      0.71      0.82       186
           1       0.78      0.98      0.87       192

    accuracy

**4. LogisticRegression**

---



In [ ]:
from sklearn.linear_model import LogisticRegression
Logistic_clf_Cv = LogisticRegression(max_iter=500)
Logistic_clf_Tfid = LogisticRegression(max_iter=500)

Logistic_clf_Cv.fit(x_train_cv_features,train_data['Label'])
Logistic_clf_Tfid.fit(x_train_tfid_features,train_data['Label'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
dump(Logistic_clf_Cv,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/Logistic_clf_Cv.joblib')
dump(Logistic_clf_Tfid,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/Logistic_clf_Tfid.joblib')

['/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/Logistic_clf_Tfid.joblib']

In [ ]:
# LogisticRegression Model prediction

Cv_predictions_4 = Logistic_clf_Cv.predict(x_test_cv_features)
Tfid_predictions_4 = Logistic_clf_Tfid.predict(x_test_tfid_features)

In [ ]:
matrix_4 = confusion_matrix(test_data['Label'],Cv_predictions_4)
report_4 = classification_report(test_data['Label'],Cv_predictions_4)
print(matrix_4)
print("----------------------------classification_report LogisticRegression Model (using bag-of-words ) -------------------------------------------------\n")
print(report_4)
print("\n------------------------------------------------------------------------------------------------------------------------------------------------\n")
matrix_4 = confusion_matrix(test_data['Label'],Tfid_predictions_4)
report_4 = classification_report(test_data['Label'],Tfid_predictions_4)
print(matrix_4)
print("----------------------------Classification Report LogisticRegression Model (using TF_ID)-----------------------------------------------------------\n")
print(report_4)

[[151  35]
 [ 27 165]]
----------------------------classification_report LogisticRegression Model (using bag-of-words ) -------------------------------------------------

              precision    recall  f1-score   support

           0       0.85      0.81      0.83       186
           1       0.82      0.86      0.84       192

    accuracy                           0.84       378
   macro avg       0.84      0.84      0.84       378
weighted avg       0.84      0.84      0.84       378


------------------------------------------------------------------------------------------------------------------------------------------------

[[130  56]
 [ 14 178]]
----------------------------Classification Report LogisticRegression Model (using TF_ID)-----------------------------------------------------------

              precision    recall  f1-score   support

           0       0.90      0.70      0.79       186
           1       0.76      0.93      0.84       192

    accuracy       

**5. DecisionTreeClassifier**

---


In [ ]:
from sklearn.tree import DecisionTreeClassifier
DecisionTree_clf_Cv = DecisionTreeClassifier(random_state=0)
DecisionTree_clf_Tfid = DecisionTreeClassifier(random_state=0)

DecisionTree_clf_Cv.fit(x_train_cv_features,train_data['Label'])
DecisionTree_clf_Tfid.fit(x_train_tfid_features,train_data['Label'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [ ]:
dump(DecisionTree_clf_Cv,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/DecisionTree_clf_Cv.joblib')
dump(DecisionTree_clf_Tfid,'/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/DecisionTree_clf_Tfid.joblib')

['/content/drive/MyDrive/Colab Notebooks/Sentiment analysis and prediction models/DecisionTree_clf_Tfid.joblib']

In [ ]:
# DecisionTreeClassifier Model prediction

Cv_predictions_5 = DecisionTree_clf_Cv.predict(x_test_cv_features)
Tfid_predictions_5 = DecisionTree_clf_Tfid.predict(x_test_tfid_features)

In [ ]:
matrix_5 = confusion_matrix(test_data['Label'],Cv_predictions_5)
report_5 = classification_report(test_data['Label'],Cv_predictions_5)
print(matrix_5)
print("----------------------------classification_report DecisionTreeClassifier Model (using bag-of-words ) ---------------------------------------------\n")
print(report_5)
print("\n------------------------------------------------------------------------------------------------------------------------------------------------\n")
matrix_5 = confusion_matrix(test_data['Label'],Tfid_predictions_5)
report = classification_report(test_data['Label'],Tfid_predictions_5)
print(matrix_5)
print("----------------------------Classification Report DecisionTreeClassifier Model (using TF_ID)-------------------------------------------------------\n")
print(report_5)

[[162  24]
 [ 32 160]]
----------------------------classification_report DecisionTreeClassifier Model (using bag-of-words ) ---------------------------------------------

              precision    recall  f1-score   support

           0       0.84      0.87      0.85       186
           1       0.87      0.83      0.85       192

    accuracy                           0.85       378
   macro avg       0.85      0.85      0.85       378
weighted avg       0.85      0.85      0.85       378


------------------------------------------------------------------------------------------------------------------------------------------------

[[153  33]
 [ 34 158]]
----------------------------Classification Report DecisionTreeClassifier Model (using TF_ID)-------------------------------------------------------

              precision    recall  f1-score   support

           0       0.84      0.87      0.85       186
           1       0.87      0.83      0.85       192

    accuracy       